In [5]:
# CMSC426 Project 3
# GROUP MEMBERS: Andrew Sumner (asumner1), Teimuraz Trapaidze (ttrapaid)
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
from scipy import ndimage
import math
import random
import sklearn as sk
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# CONVERT IMAGES TO GRAYSCALE?????

In [6]:
folders = ["images/dolphin", "images/airplanes", "images/Leopards"]

def get_imgs_from_folders(folders):
    # Return to this
    # This must be all images, not just dolphin images, and we must track the label
    image_sets = []
    for folder in folders:
        images = []
        i = 0
        for filename in os.listdir(folder):
            i = i + 1
            image = img.imread(os.path.join(folder,filename))
            images.append(image)
            if i >= 65:
                break
        image_sets.append(images)
    return image_sets

all_images = get_imgs_from_folders(folders)

images = all_images[0] # REMOVE THIS LATER!!!!!!!!!!!!!!!!
#print(images)

# ensure feature imbalance is taken care of

In [7]:
# splitting train and test
# note that we already know the labels since the folders are still
# separated within the output data structure (for now)
train_subsets = []
test_subsets = []
for img_set in all_images:
    train_x, test_x = train_test_split(img_set, test_size=0.1)
    train_subsets.append(train_x)
    test_subsets.append(test_x)
    
#print(len(test_subsets[0]))

In [8]:
# supposedly we get descriptors for all images in a category, then 
# stack ? 
desc_all_imgs = []
kp_all_imgs = []
min_num_kp = 999999999999999

# FIRST LOOP FINDS MIN KP BOUND
for img_set in all_images:
    for image in img_set:
        sift = cv2.SIFT_create()
        kp, descriptors = sift.detectAndCompute(image, None)
        if len(kp) < min_num_kp:
            min_num_kp = len(kp)

print("MIN NUM KP = ", min_num_kp)
# SECOND LOOP FINDS BEST KEYPOINTS WHERE ALL IMGS HAVE SAME (MIN) NUM
for img_set in all_images:
    for image in img_set:
        sift = cv2.SIFT_create(nfeatures=min_num_kp)
        kp, descriptors = sift.detectAndCompute(image, None)
        #TRIM DESCRIPTORS HERE
        print(len(descriptors))
        desc_all_imgs.append(descriptors)
        kp_all_imgs = kp_all_imgs + kp

#     print(descriptors.shape)
#     for pt, desc in zip(kp, descriptors):
#         (x,y) = pt.pt
#         print((x,y), desc)

desc_all_imgs = tuple(desc_all_imgs)
desc_all_imgs = np.vstack(desc_all_imgs)
#print(desc_all_imgs)
print()
print(desc_all_imgs.shape)

MIN NUM KP =  34
34
34
34
34
34
36
34
34
34
34
34
35
34
34
34
35
35
34
34
34
34
34
34
34
34
34
34
34
34
36
34
35
34
34
35
34
34
35
34
35
34
34
34
34
34
34
34
34
34
34
34
35
34
34
34
34
34
34
35
34
34
34
34
35
34
34
34
35
35
34
34
34
34
35
34
34
34
34
34
35
34
34
34
34
35
34
34
34
34
34
34
34
35
34
34
34
34
34
34
35
35
34
34
35
34
34
34
34
34
35
35
34
35
34
34
34
34
35
34
34
34
34
35
34
34
35
34
34
34
34
35
34
34
34
34
34
34
34
34
34
34
34
34
34
35
34
34
34
35
34
34
35
34
34
34
34
34
35
34
34
34
34
34
34
34
34
35
34
34
34
34
34
34
34
34
35
34
34
36
34
34
34
34
35
35
36
34
34
34
34
34
35
34
35
34

(6674, 128)


In [132]:
def clustering(keypoints, descriptors, k):
    num_descs = len(keypoints)
    
    center_descs = []
    # initial random k centers
    for i in range(k):
        center = random.randint(0, num_descs-1)
        center_descs.append(descriptors[center])
        
    check = False
    
    while (not check):
        clusters = {i:[] for i in range(k)}
            
        for descriptor in descriptors:
            closest = 99999999999
            i = 0
            for center_desc in center_descs:
#                 dist = np.sum((center_desc-descriptor)**2)

                dist = np.linalg.norm(center_desc - descriptor)
                if dist < closest:
                    closest = dist
                    closest_center = i
                i += 1

            clusters[closest_center].append(descriptor)
            
        
#         for key, value in clusters.items():
#             print(key, len(value))
                
#         print("\n\n")
        
        old_centers = center_descs
        center_descs = []
        
        print(len(old_centers))
        
        for center_num, cluster in clusters.items():
            if (len(cluster) > 0):
                cluster = np.vstack(cluster)
                new_center = np.sum(cluster, axis=0)/np.linalg.norm(cluster)
                center_descs.append(new_center)
        
        check = True
        for old, new in zip(old_centers, center_descs):
            if not np.array_equal(old, new):
                check = False
                break
        
        
    return center_descs

In [133]:
# i havent checked this yet, but if you were to sum the number of key 
# points in each cluster, it should equal the number of descriptors 

# this is for one image

# issue: why is the number of clusters decreasing with each iteration

centers = clustering(kp_all_imgs, desc_all_imgs, 400)

400
382
193
74
22
3
1


In [134]:
print(centers)

[array([ 6149.1064,  4379.013 ,  3884.8135,  3795.2483,  4707.0557,
        3304.2722,  2870.352 ,  3501.7734, 16653.69  ,  7411.3667,
        4594.0312,  4052.2136,  5321.038 ,  4067.711 ,  4378.5083,
        7916.0537, 14907.638 ,  6536.7295,  3713.6843,  3844.808 ,
        6236.7295,  5107.3403,  5534.58  ,  8050.129 ,  5575.2456,
        3180.528 ,  2567.8274,  3237.7004,  5179.3887,  4108.5703,
        4314.0728,  4681.1494,  9375.66  ,  4822.838 ,  4690.8594,
        5341.5845,  6624.0415,  4589.565 ,  3512.6487,  4338.542 ,
       25114.512 ,  8782.263 ,  6607.6514,  7979.6733,  8962.092 ,
        6228.6895,  5972.268 , 12614.914 , 22628.016 , 10447.993 ,
        5091.921 ,  6150.0776, 10965.147 ,  9869.316 ,  7934.5415,
        9681.33  ,  8084.6577,  3900.5435,  3258.2468,  4888.5547,
        7647.3975,  6074.417 ,  5255.36  ,  4887.3896,  9441.145 ,
        4226.2944,  3370.2224,  4619.899 ,  6684.865 ,  5332.224 ,
        4822.3716,  4879.0005, 25093.615 , 12480.178 ,  5848.

In [ ]:
def generate_histogram(descriptors, centers): # descriptors is for a single image
    hist_arr = []
    num_bins = len(centers)
    for row in descriptors:
        min_dist = 9999999999999999999
        min_center_idx = -1
        center_idx = -1
        for center in centers:
            center_idx = center_idx + 1
            dist = np.linalg.norm(row - center)
            if dist < min_dist:
                min_dist = dist
                min_center_idx = center_idx
        hist_arr.append(min_center_idx)
    hist = np.histogram(hist_arr, bins=num_bins)
    plt.hist(hist_arr, bins=num_bins)
    plt.show()
    return hist

In [ ]:
# svm

In [ ]:
# use this later once we have our test and train cleaned up

svclassifier = sk.svm.SVC(kernel='linear')
svclassifier.fit(x_train, y_train)
y_test_pred = svclassifier.predict(x_test)

print(sk.metrics.classification_report(y_test, y_test_pred))